## This notebook is intended to train a LSTM model using the different data generated for different classification scenarios.  

### Importing the required libraries

In [ ]:
import os
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import time
import math
from tensorflow import keras
import seaborn as sns
from sklearn import metrics
import random
import tensorflow as tf

#### Timing the training process

In [ ]:
start_time = time.time()

#### Filenames to be used for storing models, training process and the input files

In [ ]:
file_str="RWs_H_g_2_tadv_10min_deep_buoys_rw_0.5"
file_str_test = "RWs_H_g_2_tadv_10min_deep_buoys_test_rw_0.5"

LSTM_save_name= os.getcwd()+ "/model_saves" + "/" + "best_LSTM_"+file_str +"checkpoint.model.keras"
metrics_save_name = os.getcwd() + "/metric_saves_lstm" + "/" + file_str + ".txt"

#### Creating the train and test data for training the LSTM model.
**The training data for different scenarios can be accessed here.**

In [ ]:
data=np.load(file_str+".npz")
data_test = np.load(file_str_test+".npz")

for vars in data:
    print(vars)

wave_data_train=data["wave_data_train"]
wave_data_test=data_test["wave_data_test"]
label_train=data["label_train"]
label_test=data_test["label_test"]

num_classes=2

print(wave_data_train.shape)
print(wave_data_test.shape)

**Creating the LSTM model using Tensorflow. Callbacks were created to enable early stopping and learning rate schedulers were put in place to schedule the learning rate with epoch progression.**
**The different hyperparameters used here were chosen after hyperparameter tuning.**

In [ ]:
batch_size=64

model_LSTM = keras.Sequential()
model_LSTM.add(keras.layers.LSTM(10, input_shape = wave_data_train.shape[1:], return_sequences=True))
model_LSTM.add(keras.layers.BatchNormalization())

model_LSTM.add(keras.layers.LSTM(10, return_sequences=True)) #, return_sequences=True
model_LSTM.add(keras.layers.BatchNormalization())

model_LSTM.add(keras.layers.LSTM(10, return_sequences=True))#, return_sequences=True
model_LSTM.add(keras.layers.BatchNormalization())

model_LSTM.add(keras.layers.LSTM(10))
model_LSTM.add(keras.layers.BatchNormalization())
model_LSTM.add(keras.layers.Dropout(0.05))
model_LSTM.add(keras.layers.Dense(num_classes, activation="sigmoid"))#


model_LSTM.compile(loss="sparse_categorical_crossentropy",
            metrics=["sparse_categorical_accuracy"],
            optimizer="adam")

model_LSTM.summary()

import math
def scheduler(epochs, lr):
    if epochs < 5:
        return lr
    else:
        return lr * math.exp(-0.1)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        LSTM_save_name, save_best_only=True, monitor="val_loss"#"val_sparse_categorical_accuracy"#"val_loss"
    ),
   
    keras.callbacks.LearningRateScheduler(scheduler),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=25, verbose=0), #"val_loss"
]

history_LSTM=model_LSTM.fit(wave_data_train, label_train,batch_size=batch_size,   epochs=500, validation_split=0.2, callbacks=callbacks, 
          verbose=0)

**Function to generate the training curves and time the training process.**

In [ ]:
metric = "sparse_categorical_accuracy"
save_history_plot=True
if save_history_plot ==True:
    plt.figure()
    plt.plot(history_LSTM.history[metric])
    plt.plot(history_LSTM.history["val_" + metric])
    plt.title("model " + metric)
    plt.ylabel(metric, fontsize="large")
    plt.xlabel("epoch", fontsize="large")
    plt.legend(["train", "val"], loc="best")
    filename=os.getcwd()+'/training_history_'+'/'+file_str+'.jpg'
    print(filename)
    plt.savefig(filename,dpi=199)
    plt.close()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time is {elapsed_time} seconds.")

**A sample plot depicting the training and validation accuracies are plotted here.**
<img src="training_RWs_H_g_2_tadv_5min_rw_smallWindow_0.5.jpg" width="800">